In [20]:
import pathlib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import plotnine as gg
import click
import logging
from pathlib import Path
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pylab
from pylab import xticks

  

In [110]:
path_customer_churn1 = pathlib.Path("../data/raw/customer_churn_0005.csv")
customer_churn1 = pd.read_csv(path_customer_churn1, delimiter=",")
len(customer_churn1)

In [4]:
path_customer_usage1 = pathlib.Path("../data/raw/customer_usage_0005.csv")
customer_usage1 = pd.read_csv(path_customer_usage1, delimiter=",")
len(customer_usage1)

136302

In [6]:
path_customer_churn = pathlib.Path("../data/interim/customer_churn.csv")
path_customer_usage = pathlib.Path("../data/interim/customer_usage.csv")

In [7]:
customer_churn = pd.read_csv(path_customer_churn, delimiter=",")

In [8]:
customer_usage = pd.read_csv(path_customer_usage, delimiter=",")

In [9]:
customer_churn.describe(include="all")

,year,month,user_account_id,churn
count,5061.0,5061.0,5.061000e+03,5061.000000
mean,2013.0,9.0,5.937355e+05,0.205493
std,0.0,0.0,4.294465e+05,0.404101
min,2013.0,9.0,2.700000e+01,0.000000
25%,2013.0,9.0,2.453490e+05,0.000000
50%,2013.0,9.0,4.139200e+05,0.000000
75%,2013.0,9.0,1.041736e+06,0.000000
max,2013.0,9.0,1.196933e+06,1.000000


In [10]:
customer_usage.describe(include="all")

,year,month,user_account_id,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,...,last_100_reloads_sum,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage
count,13844.0,13844.000000,1.384400e+04,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,...,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000,13844.000000
mean,2013.0,7.049913,5.361070e+05,2743.831479,0.025571,42.471034,10.414656,7.306020,0.890566,0.661658,...,23.862560,109.154275,1.415873,72.973828,3.871051,112.436723,8.454565,68.706082,4.302875,9.008170
std,0.0,0.823888,4.056738e+05,4846.125488,0.157856,216.267461,14.132944,14.187962,0.312194,0.473162,...,41.177729,206.398618,8.215050,163.907837,19.135336,255.196226,34.741915,180.168112,20.052433,131.293983
min,2013.0,6.000000,2.700000e+01,0.000000,0.000000,0.000000,0.000000,-5.580000,0.000000,0.000000,...,-7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.0,6.000000,2.127060e+05,445.000000,0.000000,1.000000,0.890000,0.150000,1.000000,0.000000,...,0.000000,6.280000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2013.0,7.000000,3.957340e+05,1079.000000,0.000000,1.000000,6.110000,2.785000,1.000000,1.000000,...,12.000000,41.760000,0.000000,21.080000,0.000000,22.000000,0.000000,8.000000,0.000000,0.000000
75%,2013.0,8.000000,9.868768e+05,1963.000000,0.000000,5.000000,13.890000,8.550000,1.000000,1.000000,...,30.000000,126.327500,0.000000,78.180000,0.000000,112.000000,2.000000,60.000000,0.000000,0.000000
max,2013.0,8.000000,1.196933e+06,15947.000000,1.000000,1307.000000,218.350000,317.750000,1.000000,1.000000,...,941.060000,4129.250000,348.950000,3956.200000,984.370000,3970.000000,934.000000,3861.000000,764.000000,7671.690000


In [11]:
descVar = customer_usage1.describe(include="all").T


In [12]:
describe = customer_churn1.describe(include="all")

In [132]:
spark = (
    pyspark.sql.SparkSession
    .builder
    .appName("PySpark IO examples")
    .enableHiveSupport()
    .getOrCreate()
)

In [131]:
customer_usage2 = customer_usage1.drop(columns=['year', 'user_account_id', 'month', 'user_intake', 'calls_outgoing_to_onnet_inactive_days', 
                                                'calls_outgoing_to_offnet_inactive_days','calls_outgoing_to_abroad_inactive_days', 'sms_outgoing_to_onnet_inactive_days', 'sms_outgoing_to_offnet_inactive_days', 
                                                'sms_outgoing_to_abroad_inactive_days', 'sms_incoming_from_abroad_spendings','gprs_session_count', 
                                                'gprs_usage', 'gprs_spendings' ])

In [48]:


for col in customer_usage2.columns:
    path = "../reports/figures/" + col + ".png"
    q1 = customer_usage2.quantile(.02)[col]
    q2 = customer_usage2.quantile(.98)[col]
    
    
    hist = gg.ggplot(customer_usage1, gg.aes(x=col)) + gg.geom_histogram() + gg.scales.xlim(q1, q2)
    #print(age_hist + gg.theme_bw())
    hist.save(path)
 

C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: ../reports/figures/user_lifetime.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\stats\stat_bin.py:93: PlotnineWarning: 'stat_bin()' using 'bins = 270'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:360: PlotnineWarning: stat_bin : Removed 2714 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:452: PlotnineWarning: geom_histogram : Removed 2 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine

C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:360: PlotnineWarning: stat_bin : Removed 1893 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:452: PlotnineWarning: geom_histogram : Removed 2 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: ../reports/figures/reloads_sum.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\stats\stat_bin.py:93: PlotnineWarning: 'stat_bin()' using 'bins = 103'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\l

C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\stats\stat_bin.py:93: PlotnineWarning: 'stat_bin()' using 'bins = 370'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:360: PlotnineWarning: stat_bin : Removed 2722 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:452: PlotnineWarning: geom_histogram : Removed 2 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: ../reports/figures/calls_outgoing_to_onnet_duration.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\sit

C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: ../reports/figures/sms_outgoing_spendings.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\stats\stat_bin.py:93: PlotnineWarning: 'stat_bin()' using 'bins = 228'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:360: PlotnineWarning: stat_bin : Removed 2730 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:452: PlotnineWarning: geom_histogram : Removed 2 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
C:\Users\edgad\Anaconda3\lib\site-packages

C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:452: PlotnineWarning: geom_histogram : Removed 2 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: ../reports/figures/sms_incoming_count.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\stats\stat_bin.py:93: PlotnineWarning: 'stat_bin()' using 'bins = 93'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:360: PlotnineWarning: stat_bin : Removed 2717 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
C:\Users\edgad\Anaconda3\lib\site-packages\plot

C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\stats\stat_bin.py:93: PlotnineWarning: 'stat_bin()' using 'bins = 163'. Pick better value with 'binwidth'.
  warn(msg.format(params['bins']), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:360: PlotnineWarning: stat_bin : Removed 2727 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\layer.py:452: PlotnineWarning: geom_histogram : Removed 2 rows containing missing values.
  self.data = self.geom.handle_na(self.data)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\edgad\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: ../reports/figures/last_100_calls_outgoing_to_abroad_duration.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\edgad\Anacond

In [112]:
q = customer_usage1.quantile([.05, .95]).T
corr_usage1 = customer_usage2.corr(method ='pearson') 
specific_dir = "../data/output/zvalgomoji/"
q.to_csv(specific_dir+'/VariablesQuantiles.csv')
corr_usage1.to_csv(specific_dir+'/VariablesCorr.csv')

In [113]:
descVar.to_csv(specific_dir+'/VariablesDesc.csv')

In [91]:
import operator
import functools

import pyspark
import pandas as pd
import plotnine as gg
import IPython

import pyspark.sql.functions as sql_fn
from pyspark.ml import stat
from pyspark.ml import feature
from pyspark.sql.functions import col, countDistinct

In [114]:
specific_dir = "../data/output/zvalgomoji/"
customer_usage2.to_csv(specific_dir+'/dataspark.csv')

In [134]:
specific_dir = "../data/output/zvalgomoji/"
customer_usage1.to_csv(specific_dir+'/datasparkVisas.csv')

In [135]:
customer_usage21 = spark.read.csv(
    "../data/output/zvalgomoji/dataspark.csv", 
    inferSchema=True, 
    header=True)

In [136]:
customer_usage1Visas = spark.read.csv(
    "../data/output/zvalgomoji/datasparkVisas.csv", 
    inferSchema=True, 
    header=True)

In [137]:
column_quantiles = customer_usage1Visas.columns

In [138]:
probabilities = [0.02, 0.95, 0.98]
relative_error_quantile = 0.01

column_quantiles= [
    (column, customer_usage1Visas.approxQuantile(column, probabilities, relative_error_quantile),)
    for column in quantile_columns
]

In [139]:
column_quantiles_98 = [
    (column, quantiles[-1],)
    for column, quantiles
    in column_quantiles
]

## suranda tarp 2 ir 98 kvantilio duomenis

In [140]:
quantile_filters = [
    customer_usage1Visas[column] <= quantile
    for column, quantile
    in column_quantiles_98
]



In [141]:
merged_quantile_filter = functools.reduce(operator.and_, quantile_filters)



In [142]:
filtered_customer_usage21 = customer_usage1Visas.where(merged_quantile_filter)

In [143]:
filtered_customer_usage21.count()

96818

In [144]:
ismesti_df = customer_usage1Visas.subtract(filtered_customer_usage21)


In [145]:
ismesti_df.count()

39484

In [146]:
customer_usage1Visas.select(countDistinct("user_account_id")).show()

+-------------------------------+
|count(DISTINCT user_account_id)|
+-------------------------------+
|                          49693|
+-------------------------------+



In [ ]:
customer_usage1Visas = customer_usage1Visas.join(ismesti_df, ("user_account_id"),'left_anti')
customer_usage1Visas.select(countDistinct("user_account_id")).show()

In [129]:
customer_usage1Visas = customer_usage1Visas[customer_usage1Visas['user_lifetime']<10000]

In [130]:
merged_quantile_filter2 = functools.reduce(operator.and_, customer_usage1Visas)


In [109]:
filtered_customer_usage22 = customer_usage1Visas.where(merged_quantile_filter2)

AnalysisException: "cannot resolve '(`user_account_id` AND `_c0`)' due to data type mismatch: '(`user_account_id` AND `_c0`)' requires boolean type, not int;;\n'Filter (((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((user_account_id#3767 && _c0#3764) && year#3765) && month#3766) && user_lifetime#3768) && user_intake#3769) && user_no_outgoing_activity_in_days#3770) && user_account_balance_last#3771) && user_spendings#3772) && user_has_outgoing_calls#3773) && user_has_outgoing_sms#3774) && user_use_gprs#3775) && user_does_reload#3776) && reloads_inactive_days#3777) && reloads_count#3778) && reloads_sum#3779) && calls_outgoing_count#3780) && calls_outgoing_spendings#3781) && calls_outgoing_duration#3782) && calls_outgoing_spendings_max#3783) && calls_outgoing_duration_max#3784) && calls_outgoing_inactive_days#3785) && calls_outgoing_to_onnet_count#3786) && calls_outgoing_to_onnet_spendings#3787) && calls_outgoing_to_onnet_duration#3788) && calls_outgoing_to_onnet_inactive_